In [18]:
from hashlib import sha256
from base64 import urlsafe_b64encode
import json

### SD-JWT
An Selective Disclosure JWT (SD-JWT) is a digitally signed JSON document containing hash digests over the claim values with unique random salts and other metadata.

To calculate the digest values, a hash function is used over the disclosures. Each disclosure consists of the claim name, the claim value, and a random salt. The hash function takes these components into account to generate a digest value for each disclosure. 

Disclosures refer to the information or claims that are selectively revealed or disclosed within the token. Each disclosure consists of the claim name, the claim value, and a random salt.

- Claim Name: It specifies the name or identifier of the claim being disclosed. Examples of claim names could be "email", "birthdate", "address", etc.

- Claim Value: It represents the actual value associated with the disclosed claim. For instance, if the claim name is "email", the claim value might be the email address of the token holder.

- Random Salt: It is a random value or string that is included in the disclosure. The salt is used as part of the hash function calculation to ensure uniqueness and add an additional layer of security to the digest value.

By including disclosures in the SD-JWT, users can selectively share specific claim information without revealing all the details within the token, thereby maintaining privacy and control over their personal information.

In [19]:
# https://github.com/oauth-wg/oauth-selective-disclosure-jwt/blob/master/sd_jwt/operations.py

def toJson(data):
    return json.dumps(data)


def base64encode(data):
    return urlsafe_b64encode(data).decode("ascii").strip("=")


def create_sdjwt_disclosure(salt, claim_name, claim_value):
    data = toJson([salt, claim_name, claim_value]).encode('utf-8')
    return base64encode(data)


In [20]:
disclosure = create_sdjwt_disclosure('_26bc4LT-ac6q2KI6cBW5es', 'family_name', 'Möbius')
# check with SD-JWT paper
result = 'WyJfMjZiYzRMVC1hYzZxMktJNmNCVzVlcyIsICJmYW1pbHlfbmFtZSIsICJNXHUwMGY2Yml1cyJd'
result == disclosure

True

The digest is generated by applying a hash function to the contents of the disclosure.
The purpose of the digest is to ensure that the disclosed claim has not been tampered with or modified during transit. By comparing the digest value included in the token with the recalculated digest of the received disclosure, the recipient can verify that the claim information remains intact and unchanged.

In [21]:
def create_sdjwt_digest(disclosure):
    return base64encode(sha256(disclosure).digest())

In [22]:
digest = create_sdjwt_digest('WyI2cU1RdlJMNWhhaiIsICJmYW1pbHlfbmFtZSIsICJNw7ZiaXVzIl0'.encode("ascii"))
# check with SD-JWT paper
result = 'uutlBuYeMDyjLLTpf6Jxi7yNkEF35jdyWMn9U7b_RYY'
result == digest

True

Disclosure and hash digest creates the SD-JWT claim

In [23]:
def create_sdjwt_claim(salt, claim_name, claim_value):
    disclosure = create_sdjwt_disclosure(salt, claim_name, claim_value)
    hash_digest = create_sdjwt_digest(disclosure.encode("ascii"))
    return hash_digest, disclosure


# overload method to handle list directly
def generate_sdjwt_claim(contentlist):
    return create_sdjwt_claim(contentlist[0], contentlist[1], contentlist[2])

In [24]:
# check with SD-JWT paper
result_discl = 'WyJRUGtibHhUbmJTTEw5NEkyZlpJYkhBIiwgImxvY2FsaXR5IiwgIlNjaHVscGZvcnRhIl0'
result_diges = 'KlG6HEM6XWbymEJDfyDY4klJkQQ9iTuNG0LQXnE9mQ0'
digest, disclosure = create_sdjwt_claim('QPkblxTnbSLL94I2fZIbHA', 'locality', 'Schulpforta')

result_discl == disclosure and result_diges == digest

True

## SD-JWT Examples
example directly from the paper

### Option 1: Flat SD-JWT

In [25]:
result_disclosure_ = 'WyJpbVFmR2oxX00wRWw3NmtkdmY3RGF3IiwgImFkZHJlc3MiLCB7InN0cmVldF9hZGRyZXNzIjogIlNjaHVsc3RyLiAxMiIsICJsb2NhbGl0eSI6ICJTY2h1bHBmb3J0YSIsICJyZWdpb24iOiAiU2FjaHNlbi1BbmhhbHQiLCAiY291bnRyeSI6ICJERSJ9XQ'
result_hash_digest = 'FphFFpj1vtr0rpYK-14fickGKMg3zf1fIpJXxTK8PAE'
digest, disclosure = generate_sdjwt_claim(["imQfGj1_M0El76kdvf7Daw", "address", {"street_address": "Schulstr. 12", "locality": "Schulpforta", "region": "Sachsen-Anhalt","country": "DE"}])
assert(result_disclosure_ == disclosure)
assert(result_hash_digest == digest)

### Option 2: Structured SD-JWT

In [26]:
# Disclosure for street_address:
result_disclosure_ = 'WyJRU05JaHVfbjZhMXJJOF8yZU5BUkNRIiwgInN0cmVldF9hZGRyZXNzIiwgIlNjaHVsc3RyLiAxMiJd'
result_hash_digest = 'G_FeM1D-U3tDJcHB7pwTNEElLal9FE9PUs0klHgeM1c'
digest, disclosure = generate_sdjwt_claim(["QSNIhu_n6a1rI8_2eNARCQ", "street_address", "Schulstr. 12"])
assert(result_disclosure_ == disclosure)
assert(result_hash_digest == digest)


# Disclosure for locality:
result_disclosure_ = 'WyJRUGtibHhUbmJTTEw5NEkyZlpJYkhBIiwgImxvY2FsaXR5IiwgIlNjaHVscGZvcnRhIl0'
result_hash_digest = 'KlG6HEM6XWbymEJDfyDY4klJkQQ9iTuNG0LQXnE9mQ0'
digest, disclosure = generate_sdjwt_claim(["QPkblxTnbSLL94I2fZIbHA", "locality", "Schulpforta"])
assert(result_disclosure_ == disclosure)
assert(result_hash_digest == digest)


# Disclosure for region:
result_disclosure_ = 'WyJqUi1ZZWQwOEFFbzRnY29ncFQ1X1VBIiwgInJlZ2lvbiIsICJTYWNoc2VuLUFuaGFsdCJd'
result_hash_digest = 'ffPGyxFBnNA1r60g2f796Hqq3dBGtaOogpnIBgRGdyY'
digest, disclosure = generate_sdjwt_claim(["jR-Yed08AEo4gcogpT5_UA", "region", "Sachsen-Anhalt"])
assert(result_disclosure_ == disclosure)
assert(result_hash_digest == digest)


# Disclosure for country:
result_disclosure_ = 'WyJ5djNBblV6ODgtQUx3VjhISld6bi1nIiwgImNvdW50cnkiLCAiREUiXQ'
result_hash_digest = 'X96Emv4S9uzFUGkU8MmOlFzUwEtDNeT-ToXw3Fx9AfI'
digest, disclosure = generate_sdjwt_claim(["yv3AnUz88-ALwV8HJWzn-g", "country", "DE"])
assert(result_disclosure_ == disclosure)
assert(result_hash_digest == digest)

### Option 3: SD-JWT with Recursive Disclosures

In [27]:
# Disclosure for street_address:
result_disclosure_ = 'WyIyUzBkSXhpSE9CblVldFBKMW5GbERBIiwgInN0cmVldF9hZGRyZXNzIiwgIlNjaHVsc3RyLiAxMiJd'
result_hash_digest = 'udFWFx_91NgCjhLTHJAvvAHwNT2vlHBsWxPIOxK2Ydo'
digest, disclosure = generate_sdjwt_claim(["2S0dIxiHOBnUetPJ1nFlDA", "street_address", "Schulstr. 12"])
assert(result_disclosure_ == disclosure)
assert(result_hash_digest == digest)


# Disclosure for locality:
result_disclosure_ = 'WyI5YU9BQzFHNHZlNFJjalJvTUtERWNnIiwgImxvY2FsaXR5IiwgIlNjaHVscGZvcnRhIl0'
result_hash_digest = '1zgsu9Fe6bkbQY9zJsfPTo1uBPMVl48m4EafxJTYSu0'
digest, disclosure = generate_sdjwt_claim(["9aOAC1G4ve4RcjRoMKDEcg", "locality", "Schulpforta"])
assert(result_disclosure_ == disclosure)
assert(result_hash_digest == digest)


# Disclosure for region:
result_disclosure_ = 'WyJqdkZ5QUxBaGFaa0hBU0tCYUZqMGJBIiwgInJlZ2lvbiIsICJTYWNoc2VuLUFuaGFsdCJd'
result_hash_digest = 'DKtN_qW_hayYCGCKihS39M3sobs1RTekxaKnMbE17-Q'
digest, disclosure = generate_sdjwt_claim(["jvFyALAhaZkHASKBaFj0bA", "region", "Sachsen-Anhalt"])
assert(result_disclosure_ == disclosure)
assert(result_hash_digest == digest)


# Disclosure for country:
result_disclosure_ = 'WyI0dGNmbFJNZWdxUG54ZzZTUkQ2ZzBnIiwgImNvdW50cnkiLCAiREUiXQ'
result_hash_digest = 'h2DxxE80C7qRWxCQjwR96da46jr2B8bLSRwAdbZSRlo'
digest, disclosure = generate_sdjwt_claim(["4tcflRMegqPnxg6SRD6g0g", "country", "DE"])
assert(result_disclosure_ == disclosure)
assert(result_hash_digest == digest)

In [28]:
# is disclosure right format
result_hash_digest = 'Lot9gM9VOOiyFS4FOR1ykwz-V-fB_xwF63jWemIH4d0'
result_disclosure_ = "WyJGcDBqLTd6Q2FOVFdmX3dLNjdzeGlnIiwgImFkZHJlc3MiLCB7Il9zZCI6IFsiMXpnc3U5RmU2YmtiUVk5ekpzZlBUbzF1QlBNVmw0OG00RWFmeEpUWVN1MCIsICJES3ROX3FXX2hheVlDR0NLaWhTMzlNM3NvYnMxUlRla3hhS25NYkUxNy1RIiwgImgyRHh4RTgwQzdxUld4Q1Fqd1I5NmRhNDZqcjJCOGJMU1J3QWRiWlNSbG8iLCAidWRGV0Z4XzkxTmdDamhMVEhKQXZ2QUh3TlQydmxIQnNXeFBJT3hLMllkbyJdfV0"
digest = create_sdjwt_digest(result_disclosure_.encode("ascii"))
assert(result_hash_digest == digest)

In [29]:
# claim contents selectively disclosable recursively
digest, disclosure = generate_sdjwt_claim(["Fp0j-7zCaNTWf_wK67sxig", "address", {"_sd": ["1zgsu9Fe6bkbQY9zJsfPTo1uBPMVl48m4EafxJTYSu0","DKtN_qW_hayYCGCKihS39M3sobs1RTekxaKnMbE17-Q","h2DxxE80C7qRWxCQjwR96da46jr2B8bLSRwAdbZSRlo","udFWFx_91NgCjhLTHJAvvAHwNT2vlHBsWxPIOxK2Ydo"]}])
assert(result_disclosure_ == disclosure)
assert(result_hash_digest == digest)

### Example 1: SD-JWT


In [30]:
result_disclosure_ = 'WyJOYTNWb0ZGblZ3MjhqT0FyazdJTlZnIiwgImFkZHJlc3MiLCB7InN0cmVldF9hZGRyZXNzIjogIjEyMyBNYWluIFN0IiwgImxvY2FsaXR5IjogIkFueXRvd24iLCAicmVnaW9uIjogIkFueXN0YXRlIiwgImNvdW50cnkiOiAiVVMifV0'
result_hash_digest = '5nXy0Z3QiEba1V1lJzeKhAOGQXFlKLIWCLlhf_O-cmo'
digest, disclosure = generate_sdjwt_claim(["Na3VoFFnVw28jOArk7INVg", "address", {"street_address": "123 Main St", "locality": "Anytown", "region": "Anystate","country": "US"}])
assert(result_disclosure_ == disclosure)
assert(result_hash_digest == digest)

### Example 3 - Complex Structured SD-JWT
https://www.ietf.org/id/draft-ietf-oauth-selective-disclosure-jwt-04.html#name-example-3-complex-structure

In [31]:
# Disclosure for time:
result_disclosure_ = 'WyJjZllESjNFR2JFTFMyYkhsTWlrQ3FBIiwgInRpbWUiLCAiMjAxMi0wNC0yM1QxODoyNVoiXQ'
result_hash_digest = 'OSXqQur4cQzXkSlbTehtzOzsZBMgAIigvZmiNCV5Vd8'
digest, disclosure = generate_sdjwt_claim(["cfYDJ3EGbELS2bHlMikCqA", "time", "2012-04-23T18:25Z"])
assert(result_disclosure_ == disclosure)
assert(result_hash_digest == digest)


# Disclosure for document:
result_disclosure_ = 'WyJXVDhPVC05RnJwbTk5WEpQa1UtWTdRIiwgImRvY3VtZW50IiwgeyJ0eXBlIjogImlkY2FyZCIsICJpc3N1ZXIiOiB7Im5hbWUiOiAiU3RhZHQgQXVnc2J1cmciLCAiY291bnRyeSI6ICJERSJ9LCAibnVtYmVyIjogIjUzNTU0NTU0IiwgImRhdGVfb2ZfaXNzdWFuY2UiOiAiMjAxMC0wMy0yMyIsICJkYXRlX29mX2V4cGlyeSI6ICIyMDIwLTAzLTIyIn1d'
result_hash_digest = 'FPNPZmj_KaKlIP2QWB55J1gHhA14mPlbVdR9bGBgH4w'
digest, disclosure = generate_sdjwt_claim(["WT8OT-9Frpm99XJPkU-Y7Q", "document", {"type": "idcard", "issuer": {"name": "Stadt Augsburg", "country": "DE"}, "number": "53554554","date_of_issuance": "2010-03-23", "date_of_expiry": "2020-03-22"}])
assert(result_disclosure_ == disclosure)
assert(result_hash_digest == digest)


# Disclosure for nationalities:
result_disclosure_ = 'WyJQUy1fNnFqdFk0akpOSlF5SkNpeU5BIiwgIm5hdGlvbmFsaXRpZXMiLCBbIkRFIl1d'
result_hash_digest = 'd7ln7xO-RREXmecBVEpwLwrEjZ3FKs_KLhCD0vL74Us'
digest, disclosure = generate_sdjwt_claim(["PS-_6qjtY4jJNJQyJCiyNA", "nationalities", ["DE"]])
assert(result_disclosure_ == disclosure)
assert(result_hash_digest == digest)

The example demonstrate the complex structor:

In [32]:
complex_object = {
    "verified_claims": {
        "verification": {
            "trust_framework": "de_aml",
            "time": "2012-04-23T18:25Z",
            "verification_process": "f24c6f-6d3f-4ec5-973e-b0d8506f3bc7",
            "evidence": [
                {
                    "type": "document",
                    "method": "pipp",
                    "time": "2012-04-22T11:30Z",
                    "document": {
                        "type": "idcard",
                        "issuer": {
                            "name": "Stadt Augsburg",
                            "country": "DE"
                        },
                        "number": "53554554",
                        "date_of_issuance": "2010-03-23",
                        "date_of_expiry": "2020-03-22"
                    }
                }
            ]
        },
        "claims": {
            "given_name": "Max",
            "family_name": "Müller",
            "nationalities": [
                "DE"
            ],
            "birthdate": "1956-01-28",
            "place_of_birth": {
                "country": "IS",
                "locality": "Þykkvabæjarklaustur"
            },
            "address": {
                "locality": "Maxstadt",
                "postal_code": "12344",
                "country": "DE",
                "street_address": "Weidenstraße 22"
            }
        }
    },
    "birth_middle_name": "Timotheus",
    "salutation": "Dr.",
    "msisdn": "49123456789"
}


Collect all neasted properties

In [33]:
# list claims
claims_keys = list(complex_object['verified_claims']['claims'].keys())
claims_values = list(complex_object['verified_claims']['claims'].values())
claims_keys

['given_name',
 'family_name',
 'nationalities',
 'birthdate',
 'place_of_birth',
 'address']

In [34]:
# for each attribute

# Disclosure for given_name:
result_disclosure_ = 'WyJuVWtieGhxc2pyVTNaTEtVSGNBYWxBIiwgImdpdmVuX25hbWUiLCAiTWF4Il0'
result_hash_digest = 'Ylx9AxQmJxSR9CgZmFBnyKCha9qNsnA7GzrmXPBKkDo'
digest, disclosure = generate_sdjwt_claim(["nUkbxhqsjrU3ZLKUHcAalA", "given_name", claims_values[0]])
assert(result_disclosure_ == disclosure)
assert(result_hash_digest == digest)


# Disclosure for family_name:
result_disclosure_ = 'WyIyTmd0UExxQmU0QVhMVzR6SUNLWHJ3IiwgImZhbWlseV9uYW1lIiwgIk1cdTAwZmNsbGVyIl0'
result_hash_digest = 'fIW6zXRtRSQNcNK5gvpRsZFkaN8BMh36UwhsdZlxoTo'
digest, disclosure = generate_sdjwt_claim(["2NgtPLqBe4AXLW4zICKXrw", "family_name", claims_values[1]])
assert(result_disclosure_ == disclosure)
assert(result_hash_digest == digest)


# Disclosure for nationalities:
result_disclosure_ = 'WyJQUy1fNnFqdFk0akpOSlF5SkNpeU5BIiwgIm5hdGlvbmFsaXRpZXMiLCBbIkRFIl1d'
result_hash_digest = 'd7ln7xO-RREXmecBVEpwLwrEjZ3FKs_KLhCD0vL74Us'
digest, disclosure = generate_sdjwt_claim(["PS-_6qjtY4jJNJQyJCiyNA", "nationalities", claims_values[2]])
assert(result_disclosure_ == disclosure)
assert(result_hash_digest == digest)


# # Disclosure for birthdate:
result_disclosure_ = 'WyJMNExJMTRqVGFCVEZxV080MVM5TUlRIiwgImJpcnRoZGF0ZSIsICIxOTU2LTAxLTI4Il0'
result_hash_digest = 'U4P6UrikZAVE3mk82oLaPDiLX8hEXmEu9A6xO5j9mxE'
digest, disclosure = generate_sdjwt_claim(["L4LI14jTaBTFqWO41S9MIQ", "birthdate", claims_values[3]])
assert(result_disclosure_ == disclosure)
assert(result_hash_digest == digest)


# # Disclosure for place_of_birth:
result_disclosure_ = 'WyJsZ1pCenNHUUZ6bGlqRFZKa2ZCVGNRIiwgInBsYWNlX29mX2JpcnRoIiwgeyJjb3VudHJ5IjogIklTIiwgImxvY2FsaXR5IjogIlx1MDBkZXlra3ZhYlx1MDBlNmphcmtsYXVzdHVyIn1d'
result_hash_digest = 'fSw3UyxwNFf_CjYCCBYCRxZCGB-LIS35Pey2T7G_cHw'
digest, disclosure = generate_sdjwt_claim(["lgZBzsGQFzlijDVJkfBTcQ", "place_of_birth", claims_values[4]])
assert(result_disclosure_ == disclosure)
assert(result_hash_digest == digest)


# # Disclosure for address:
result_disclosure_ = 'WyI5VEFiRnplajZoNGFDSVc2dTZoWFZBIiwgImFkZHJlc3MiLCB7ImxvY2FsaXR5IjogIk1heHN0YWR0IiwgInBvc3RhbF9jb2RlIjogIjEyMzQ0IiwgImNvdW50cnkiOiAiREUiLCAic3RyZWV0X2FkZHJlc3MiOiAiV2VpZGVuc3RyYVx1MDBkZmUgMjIifV0'
result_hash_digest = 'OTog0pBGyAmwEqIhjSuf5Rq5UWTkIecAnutwB2nGUlk'
digest, disclosure = generate_sdjwt_claim(["9TAbFzej6h4aCIW6u6hXVA", "address", claims_values[5]])
assert(result_disclosure_ == disclosure)
assert(result_hash_digest == digest)

## Reference
- https://www.ietf.org/id/draft-ietf-oauth-selective-disclosure-jwt-04.html
